In [8]:
import os
import rdflib
import rdfextras
import json
import io

In [9]:
ls

datatset_journal/     music2rdf-1.3.6.jar  res.txt           tripliser/
json_results/         musicowl-master/     saxon9he.jar      xml2rdf_1.json
legacy_json/          MusicPattern.xsd*    SaxonHE9-9-1-4J/  xml_docs/
legacy_pp.py          musicxml_docs/       test_1.rdf        xsl_files/
mei_docs/             proposed_pp.py       test_1.xml
mei_transfomer.ipynb  rdf_docs/            transformer.sh*


In [10]:
# input variables
mei_directory = "mei_docs/"
xml_directory = "musicxml_docs/"
rdf_directory = "rdf_docs/"
mei2xml_xslt =  "xsl_files/musicpattern.xsl"
xml2rdf_xslt =  "xsl_files/xml2rdf3.xsl"

In [11]:
# pass mei files to transform them to xml
i = 1
for file in os.listdir(mei_directory):
    if file.endswith('.mei'):
     #print(os.path.join(mei_directory, file))
        input_file = os.path.join(mei_directory, file)
        xml_output = f'xml_docs/mei2xml_{i}.xml'
        print(xml_output)
     # transform mei files to xml through xslt 
        mei_to_xml = f'java -jar saxon9he.jar -s:{input_file} -xsl:{mei2xml_xslt} -o:{xml_output}'
        os.system(mei_to_xml)
        i = i+1

xml_docs/mei2xml_1.xml
xml_docs/mei2xml_2.xml
xml_docs/mei2xml_3.xml
xml_docs/mei2xml_4.xml
xml_docs/mei2xml_5.xml
xml_docs/mei2xml_6.xml
xml_docs/mei2xml_7.xml
xml_docs/mei2xml_8.xml
xml_docs/mei2xml_9.xml
xml_docs/mei2xml_10.xml
xml_docs/mei2xml_11.xml
xml_docs/mei2xml_12.xml
xml_docs/mei2xml_13.xml
xml_docs/mei2xml_14.xml
xml_docs/mei2xml_15.xml
xml_docs/mei2xml_16.xml
xml_docs/mei2xml_17.xml
xml_docs/mei2xml_18.xml
xml_docs/mei2xml_19.xml
xml_docs/mei2xml_20.xml


In [13]:
# pass xml files to transform them to rdf
j = 1
for file in os.listdir(xml_directory):
    if file.endswith('.xml'):
        #print(os.path.join(mei_directory, file))
        input_file = os.path.join(xml_directory, file)
        rdf_output = f'rdf_docs/xml2rdf_{j}.rdf'
        print(rdf_output)
        # transform xml files to rdf through xslt
        xml_to_rdf = f'java -jar saxon9he.jar -s:{input_file} -xsl:{xml2rdf_xslt} -o:{rdf_output}'
        os.system(xml_to_rdf)
        j = j+1

In [6]:
#from rdflib import OWL, RDF, Namespace
#MPOWL = Namespace("https://github.com/elachkarcharbel/MPOWL/blob/master/mpowl.owl")
#MPOWL.label

In [14]:
rdf_graph = rdflib.Graph()
for x in range (0,21):
    for file in os.listdir(rdf_directory):
        if file.endswith(f'_{x}.rdf'):
            input_file = os.path.join(rdf_directory, file)
            first,second = input_file.split('.')
            folder,filename = first.split('/')
            rdf_graph.parse(input_file)
           # print(rdf_graph.label)
            query_result = rdf_graph.query(
    """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?dur ?oct ?pname ?dot ?snr ?mnr ?accid ?rest
        WHERE { 
                ?oct ?dot ?dur
            }
                """)
            data = {}

            for row in query_result:
                #print(row)
                if "rdflib.term.Literal" in  str(row):
                    first,second = str(row[1]).split('score/')
                    if "/" in second:
                        measure,part,note,var2 = second.split('/')
                        if measure not in data.keys():
                            data[measure]={}
                        if note not in data[measure].keys():
                            data[measure][note]={}
                        if var2 not in data[measure][note].keys():
                            data[measure][note][var2]={}

                        data[measure][note][var2]=row[0]




            data2={}
            data2["score"]={}
            for key in sorted(data.keys()):

                for i in sorted(data[key].keys()):

                    if key not in data2["score"].keys():
                        data2["score"][key]={}
                    if i not in data2["score"][key].keys():
                        data2["score"][key][i]={}

                    data2["score"][key][i]=data[key][i]

                    if 'oct' not in data[key][i] and len(data[key][i])==1 :
                       # print(data[key][i])
                        data[key][i]['rest'] = 'true'
                      #  print(data[key][i])
                      #  print("______________________")
            #print("***",data2['score']['measure'].keys())

            json_res = json.dumps(data2, indent=4, sort_keys=True)
            with io.open(f'json_results/meires_{x}.json', 'w') as db_file:
                db_file.write(json_res)